## Data Wrangling

##### Import dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Import seaborn library and ignore warning 
sns.set_theme()
import warnings
warnings.filterwarnings('ignore')

##### Import datasets

In [ ]:
pop_df = pd.read_csv("Data/population_total.csv")
inet_df = pd.read_csv("Data/net_users_num.csv")
cell_df = pd.read_csv("Data/cell_phones_total.csv")
suic_df = pd.read_csv("Data/suicide_total_deaths.csv")
#gini_df = pd.read_csv("Data/inequality_index_gini.csv")
gdp_df = pd.read_csv("Data/gdppercapita.csv")

##### Check imported datasets

In [ ]:
pop_df.head(3)
gdp_df.head(3)
inet_df.head(3)
cell_df.head(3)
gdp_df.head(3)

*Suicide dataset contains data from 1990 to 2016; therefore I decided to perform data analysis for the years between 1990 to 2016 (included). I kept these columns using iloc method with numpy.*

##### Filtering dataset using iloc and numpy

In [ ]:
# Keep only the columns 1990 - 2016 - try to do this in the function or for loop!
# suicide dataset already sliced (1990 - 2016)
# QUESTION - is there better way of doing this 
pop_df = pop_df.iloc[:, np.r_[:1, 191:218]]
inet_df = inet_df.iloc[:, :28]
cell_df = cell_df.iloc[:, np.r_[:1, 31:58]]
#gini_df = gini_df.iloc[:, np.r_[:1, 24:51]]
gdp_df = gdp_df.iloc[:, np.r_[:1, 191:218]]

##### Handling null values

In [ ]:
# Check null values for internet and cell usage
#suic_df.isnull().sum()
#pop_df.isnull().sum()
#gdp_df.isnull().sum()
#cell_df.isnull().sum()
inet_df.isnull().sum()

*Only internet and cell datasets have null values. After examining these datasets I noticed a positive trend for all countries; therefore I decided to use `ffill` method (fill forward) instead of filling with mean or 0. I believe a better way to fill null values would be average values between empty cells. Any suggestions?*

In [ ]:
# fill first colum [1990] with null if empty 
# (I use this to fill the first column with 0. When usind ffill to populate 0 instesd of country name.)
inet_df['1990'].fillna(0, inplace=True)
cell_df['1990'].fillna(0, inplace=True)

In [ ]:
# fill null values for internet and cell phone
inet_df = inet_df.fillna(method='ffill', axis=1)
cell_df = cell_df.fillna(method='ffill', axis=1)

##### Unpivot dataframes

In [ ]:
pop_df = pop_df.melt(id_vars = 'country', var_name = 'year', value_name = 'population_total', ignore_index=True)
inet_df = inet_df.melt(id_vars = 'country', var_name = 'year', value_name = 'internet_use', ignore_index=True)
cell_df = cell_df.melt(id_vars = 'country', var_name = 'year', value_name = 'cell_use', ignore_index=True)
suic_df = suic_df.melt(id_vars = 'country', var_name = 'year', value_name = 'suicide_total', ignore_index=True)
#gini_df = gini_df.melt(id_vars = 'country', var_name = 'year', value_name = 'gini_index', ignore_index=True)
gdp_df = gdp_df.melt(id_vars = 'country', var_name = 'year', value_name = 'GDP', ignore_index=True)

*In order to compare different indicators I reshaped data with `melt` method.*

##### Merge datasets

In [ ]:
# Merge datasets two by two - please see comments below
merged01 = pd.merge(pop_df, inet_df, how="inner", on=["country", "year"])
merged02 = pd.merge(suic_df, gdp_df, how="inner", on=["country", "year"])
merged03_df = pd.merge(merged01, merged02, how="inner", on=["country", "year"])
merged_df = pd.merge(merged03_df, cell_df, how="inner", on=["country", "year"])

In [ ]:
# Check merged dataset
merged_df.head(3)

***QUESTIONS***
*Is there a better way of merging multiple datasets in one step*
*what is the best way to double-check if the merge was successful? I checked data in a spreadsheet but didn’t find this method to be found convenient, especially when working with a large dataset, the xlxs or csv file won’t load or will crash*


In [ ]:
# Export dataset for investigation - not sure if this method is good. Please see the comment above.
#merged_df.to_csv('Data/merged.csv', index=False)

##### Checking dataset (shape, null values, datatypes and duplicates) after merging

In [ ]:
# Shape
merged_df.shape

In [ ]:
# Duplicate rows
merged_df.duplicated().sum()

In [ ]:
# Missing values - decide what to do with null values
merged_df.isnull().sum()

In [ ]:
# Check individual datatypes - convert year to int
merged_df.dtypes

#### Changing datatypes

In [ ]:
# Change data types for internet use - scientific 
merged_df['internet_use'] = merged_df['internet_use'].astype(float)
merged_df['year'] = merged_df['year'].astype(int)
merged_df['cell_use'] = merged_df['cell_use'].astype(int)

In [ ]:
# Round suicide & change to integer (suicide was in float)
merged_df['suicide_total'] = merged_df.suicide_total.round()
merged_df['suicide_total'] = merged_df['suicide_total'].astype(int)

In [ ]:
# Check datatypes
merged_df.dtypes

#### Feature engeenering 

In [ ]:
# Calculate proportions of internet users, cell users, and the number of suicides % per country and year.
merged_df['internet_use_%'] = merged_df.internet_use/merged_df.population_total*100
merged_df['cell_use_%'] = merged_df.cell_use/merged_df.population_total*100
merged_df['suicide_total_%'] = merged_df.suicide_total/merged_df.population_total*100

In [ ]:
# Calculate proportions per capita (per 100,000 people) for internet users, cell users, and the number of suicides.
merged_df['internet_use_pc'] = merged_df.internet_use/merged_df.population_total*100000
merged_df['cell_use_pc'] = merged_df.cell_use/merged_df.population_total*100000
merged_df['suicide_total_pc'] = merged_df.suicide_total/merged_df.population_total*100000

In [ ]:
# Check newly created columns in whole dataframe.
merged_df.head(5)

*For a better comparison between countries, I create new columns % of total proportions per capita (per 100,000 people).*

## EDA

*In this analysis I will use columns per capita (per 100,000 people); therefore I used `iloc` method to keep only columns created in feature engineering*

In [ ]:
# Create df with columns per capita only
pc_df = merged_df.iloc[:, np.r_[:2, 5:6, 10:13]]
pc_df.head(3)

# From 1990 - 2016

## For all countries

#### FEW INVESTIGATING QUESTIONS

In [ ]:
# Country and year with most suicides per capita - used idxmax to find the row where this information is:
most_suicides_pc = pc_df.suicide_total_pc.idxmax(axis=1)
most_suicides_pc

In [ ]:
# Used iloc method to display data whit index found in the previous cell.
most_suicides_pc = pc_df.iloc[886,:]
most_suicides_pc

In [ ]:
# Find a country and year with most suicides per capita - this is another way to get the same answer as cells above.
most_suicides_pc1 = pc_df.groupby(['country','year']).mean()['suicide_total_pc'].idxmax(axis=1)
most_suicides_pc1

In [ ]:
# Year with most suicides per capita.
worst_year = pc_df.groupby(['year']).mean()['suicide_total_pc'].idxmax(axis=1)
worst_year

In [ ]:
# Country and year with least suicides per capita.
least_year = pc_df.groupby(['country','year']).mean()['suicide_total_pc'].idxmin(axis=1)
least_year

In [ ]:
# Year with least suicides per capita.
least_year = pc_df.groupby(['year']).mean()['suicide_total_pc'].idxmin(axis=1)
least_year

#### HISTOGRAM

In [ ]:
# Summary statistics
#main_df.describe()

In [ ]:
# Plotting histograms 
pc_df.hist(figsize=(10,8), color='#1f77b4');

*Explain the histograms* 

#### CORRELATION

In [ ]:
# Plotting correlation coeficient heat map.
pearsoncorr = pc_df.corr(method='pearson')
pearsoncorr
sns.heatmap(pearsoncorr, 
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5);

In [ ]:
# scatter plot matrix
pd.plotting.scatter_matrix(pc_df, figsize=(15,8));

*Explain the matrix*

In [ ]:
# Scatter plot between GDP and suicide
pc_df.plot(x='GDP', y='suicide_total_pc', kind='scatter', c='#1f77b4');

#### OUTLIERS

In [ ]:
pc_df.suicide_total_pc.plot(kind='box')

In [ ]:
pc_df.GDP.plot(kind='box')

#### SUICIDE TREND

In [ ]:
# Suicide trend (here I used acctual number of suicides (not per capita))
# What is the overall trend in suicide globally
suicide_trend_all = merged_df.groupby(['year']).sum()['suicide_total'].plot();
suicide_trend_all;

*Explant the line chart*

## Top 10 and bottom 10 countries with most/least suicides per capita

In [ ]:
### This cell is only for reference (note on the code that might be useful in the future).
## suicide_top10_ = pc_df.groupby(['country']).agg({'suicide_total_pc': ['mean','min', 'max']})
## suicide_top10_.columns =  ['s_mean','s_min', 's_max']                                          
## suicide_top10_

In [ ]:
# Top 10 countries from 1990 - 2016, get top 10 and save in a list.
suicide_top10 = pc_df.groupby(['country']).mean()['suicide_total_pc'].nlargest(10).to_frame().index.tolist()

In [ ]:
# Bottom 10 countries from 1990 - 2016, get bottom 10 and save in a list.
suicide_bottom10 = pc_df.groupby(['country']).mean()['suicide_total_pc'].nsmallest(11).to_frame().index.tolist()

In [ ]:
# Remove Kuwait (recognized as outlier = GPA >100,000)
suicide_bottom10.remove("Kuwait")

*Note: I found top 11, because during my analyis I found outlier - Kuwait, has GDP far above the average (100,000). Therefore, I excluded this country form analysis and replace it with next country in the bottom 10.*

In [ ]:
# get the data frame for top 10
top_10_countries_suicide = pc_df[pc_df['country'].isin(suicide_top10)]
top_10_countries_suicide.head(2)

In [ ]:
# get the data frame for top 10
bottom_10_countries_suicide = pc_df[pc_df['country'].isin(suicide_bottom10)]
bottom_10_countries_suicide.head(2)

### Compare top and bottom 10 countries (summary statistics, GDP, trend, internet use, cell phone use)

#### Summary statistics

In [ ]:
# Top 10
top_10_countries_suicide.describe()

In [ ]:
# Bottom 10
bottom_10_countries_suicide.describe()

*Describe summary stats*

#### GDP comparison

In [ ]:
# histogram - GDP
bottom_10_countries_suicide.GDP.hist(alpha=.7, color='green', label='bottom_10'), top_10_countries_suicide.GDP.hist(alpha=.7, color='blue', label='top_10');
# legend
plt.legend();

In [ ]:
# histogram - cell use
bottom_10_countries_suicide.cell_use_pc.hist(alpha=.7, color='green', label='bottom_10'), top_10_countries_suicide.cell_use_pc.hist(alpha=.7, color='blue', label='top_10');
# legend
plt.legend();

#### Suicide Trend

In [ ]:
# Top 10 - suicide trend
trend_top10 = top_10_countries_suicide.groupby(['year']).mean()['suicide_total_pc'].plot();
trend_top10;

In [ ]:
# Bottom 10 - suicide trend
trend_bottom10 = bottom_10_countries_suicide.groupby(['year']).mean()['suicide_total_pc'].plot();
trend_bottom10;

#### GDP Trend

In [ ]:
# Top 10 - GDP trend
trend_top10_GDP = top_10_countries_suicide.groupby(['year']).mean()['GDP'].plot();
trend_top10_GDP;

In [ ]:
# Bottom 10 - GDP trend
trend_bottom10_GDP = bottom_10_countries_suicide.groupby(['year']).mean()['GDP'].plot();
trend_bottom10_GDP;

*Explain Trend*

In [ ]:
# Acctuals countries

# From 2010 - 2016

In [ ]:
# Create df for only 2010 - 2016
main_df_reduced = merged_df.loc[merged_df.year > 2009,:]
main_df_reduced.tail()

## For all countries

In [ ]:
# Create df with columns per capita only
pc_df_reduced = main_df_reduced.iloc[:, np.r_[:2, 5:6, 10:13]]
pc_df_reduced.head(3)

#### HISTOGRAM

In [ ]:
# Plotting histograms 
pc_df_reduced.hist(figsize=(10,8), color='#1f77b4');

#### CORRELATION

In [ ]:
# Plotting correlation coeficient heat map.
pearsoncorr_reduced = pc_df_reduced.corr(method='pearson')
pearsoncorr_reduced
sns.heatmap(pearsoncorr_reduced, 
            xticklabels=pearsoncorr_reduced.columns,
            yticklabels=pearsoncorr_reduced.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5);

*Explain the matrix*

In [ ]:
# scatter plot matrix
pd.plotting.scatter_matrix(pc_df_reduced, figsize=(15,8));

In [ ]:
# Scatter plot between GDP and suicide
pc_df_reduced.plot(x='GDP', y='suicide_total_pc', kind='scatter', c='#1f77b4');

#### SUICIDE TREND

In [ ]:
# Suicide trend (here I used acctual number of suicides (not per capita))
# What is the overall trend in suicide globally
suicide_trend_all_reduced = main_df_reduced.groupby(['year']).sum()['suicide_total'].plot();
suicide_trend_all_reduced;

*Explain the line chart*

## Top 10 and bottom 10 countries with most/least suicides per capita

In [ ]:
# Top 10 countries from 2010 - 2016, get top 10 and save in a list.
suicide_top10_reduced = pc_df_reduced.groupby(['country']).mean()['suicide_total_pc'].nlargest(10).to_frame().index.tolist()

In [ ]:
# Top 10 countries from 2010 - 2016, get top 10 and save in a list.
suicide_bottom10_reduced = pc_df_reduced.groupby(['country']).mean()['suicide_total_pc'].nsmallest(11).to_frame().index.tolist()

In [ ]:
# Remove Kuwait (recognized as outlier = GPA >100,000)
suicide_bottom10_reduced.remove("Kuwait")

*Note: I found top 11, because during my analyis I found outlier - Kuwait, has GDP far above the average (100,000). Therefore, I excluded this country form analysis and replace it with next country in the bottom 10.*

In [ ]:
# get the data frame for top 10
top_10_countries_suicide_reduced = pc_df_reduced[pc_df_reduced['country'].isin(suicide_top10)]
top_10_countries_suicide_reduced.head(2)

In [ ]:
# get the data frame for top 10
bottom_10_countries_suicide_reduced = pc_df_reduced[pc_df_reduced['country'].isin(suicide_bottom10)]
bottom_10_countries_suicide_reduced.head(2)

### Compare top and bottom 10 countries (summary statistics, GDP, trend, internet use, cell phone use)

#### Summary statistics

In [ ]:
# Top 10
top_10_countries_suicide_reduced.describe()

In [ ]:
# Bottom 10
bottom_10_countries_suicide_reduced.describe()

*Explain summary statistics*

#### GDP comparison

In [ ]:
# histogram - GDP
bottom_10_countries_suicide_reduced.GDP.hist(alpha=.7, color='green', label='bottom_10'), top_10_countries_suicide_reduced.GDP.hist(alpha=.7, color='blue', label='top_10');
# legend
plt.legend();

In [ ]:
# histogram - cell use
bottom_10_countries_suicide_reduced.cell_use_pc.hist(alpha=.7, color='green', label='bottom_10'), top_10_countries_suicide_reduced.cell_use_pc.hist(alpha=.7, color='blue', label='top_10');
# legend
plt.legend();

#### Suicide Trend

In [ ]:
# Top 10 - suicide trend
trend_top10_reduced = top_10_countries_suicide_reduced.groupby(['year']).mean()['suicide_total_pc'].plot();
trend_top10_reduced;

In [ ]:
# Bottom 10 - suicide trend
trend_bottom10_reduced = bottom_10_countries_suicide_reduced.groupby(['year']).mean()['suicide_total_pc'].plot();
trend_bottom10_reduced;

#### GDP Trend

In [ ]:
# Top 10 - GDP trend
trend_top10_GDP_reduced = top_10_countries_suicide_reduced.groupby(['year']).mean()['GDP'].plot();
trend_top10_GDP_reduced;

In [ ]:
# Bottom 10 - GDP trend
trend_bottom10_GDP_reduced = bottom_10_countries_suicide_reduced.groupby(['year']).mean()['GDP'].plot();
trend_bottom10_GDP_reduced;

### GDP comparrison for top 10 and bottom 10

In [ ]:
# trend of top 10 - GDP
trend_bottom10_GDP = bottom_10_countries_suicide.groupby(['year']).mean()['GDP'].plot();
trend_bottom10_GDP

In [ ]:
# trend of top 10 - cell
trend_bottom10_cell = bottom_10_countries_suicide.groupby(['year']).mean()['cell_use_pc'].plot();
trend_bottom10_cell

In [ ]:
# trend of top 10 and bottom 10
trend_top10 = top_10_countries_suicide.groupby(['year']).mean()['suicide_total_pc'].plot();
trend_top10

In [ ]:
# trend of top 10- GDP
trend_top10_GDP = top_10_countries_suicide.groupby(['year']).mean()['GDP'].plot();
trend_top10_GDP

In [ ]:
# trend of top 10- cell
trend_top10_cell = top_10_countries_suicide.groupby(['year']).mean()['cell_use_pc'].plot();
trend_top10_cell

In [ ]:
# country totals
suicide_totals = less_years.groupby(['country', 'year']).sum()['suicide_total_pc']
suicide_totals

In [ ]:
suicide_totals.describe()

In [ ]:
# What is the difference in GDP in countries with the most suicides per capita vs least suicides per capita
# Calculate the average GDP for last 5 years
x_axis = top_10_countries_suicide.country
y_axis = top_10_countries_suicide.GDP
plt.yticks(np.arange(0, 90000, step=10000))
plt.xticks(rotation = 45)
plt.bar(x_axis, y_axis, label='GDP')
plt.legend()

In [ ]:
# What is the difference in GDP in countries with the most suicides per capita vs least suicides per capita
# Calculate the average GDP for last 5 years
x_axis = bottom_10_countries_suicide.country
y_axis = bottom_10_countries_suicide.GDP
plt.yticks(np.arange(0, 90000, step=10000))
plt.xticks(rotation = 90)
plt.bar(x_axis, y_axis, label='GDP')
plt.legend()

In [ ]:
x_axis = top_10_countries_suicide.country
y_axis = top_10_countries_suicide.suicide_total_pc
plt.xticks(rotation = 45)
plt.bar(x_axis, y_axis, label='suicide_total_pc')
plt.legend()

In [ ]:
x_axis = bottom_10_countries_suicide.country
y_axis = bottom_10_countries_suicide.suicide_total_pc
plt.xticks(rotation = 90)
plt.bar(x_axis, y_axis, label='suicide_total_pc')
plt.legend()

# From 1990 - 2000

In [ ]:
# Create df for only 1990 - 2000
main_df_reduced_cell = merged_df.loc[merged_df.year < 2001,:]
main_df_reduced_cell.head()

In [ ]:
# Create df with columns per capita only
pc_df_reduced_cell = main_df_reduced_cell.iloc[:, np.r_[:2, 5:6, 10:13]]
pc_df_reduced_cell.head(3)

In [ ]:
# Plotting correlation coeficient heat map.
pearsoncorr_reduced_cell = pc_df_reduced_cell.corr(method='pearson')
pearsoncorr_reduced_cell
sns.heatmap(pearsoncorr_reduced_cell, 
            xticklabels=pearsoncorr_reduced_cell,
            yticklabels=pearsoncorr_reduced_cell,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5);